In [1]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
from keras import metrics
from keras.constraints import NonNeg
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

import os
from os.path import join
import sys

import random

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

### https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed ###
### https://keras.io/examples/keras_recipes/reproducibility_recipes/ ###
SEED = 42
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

tf.compat.v1.disable_eager_execution()

2024-08-26 13:23:47.271629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:47.304613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:47.306991: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:47.310220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Running on GPU


## Understanding LRP methods using a simple classification NN

In [2]:
N = 10000
X = np.random.rand(N,2)
y = np.dot(X,np.array([1,2]))
oneHot = np.zeros((y.shape[0],2))
oneHot[:,0] = (y >= 3)
oneHot[:,1] = (y < 3)

In [3]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 2         , 'activation': 'softmax', 'use_bias': True, 'bias_constraint': NonNeg()}]
Losses = [{'kind': 'categorical_crossentropy', 'weight': 1.0}]

NNkwargs = {'losses': Losses, 'metrics': ['mae'],
            'batch_size': 128, 'epochs': 100, 'validation_split': 0.2,
            'filename': 'model_simpleTests_LRP', 'dirname': os.path.abspath(''),
            'random_nn_seed': 42}

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.5
    epochs_drop = 25
    lrate = initial_lrate * drop**np.floor((1+epoch)/epochs_drop)
    return lrate

keras.backend.clear_session()
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
NNkwargs['optimizer'] = sgd

K = TrainFullyConnectedNN(X, oneHot, layers = Layers, **NNkwargs)
best_model = K.quickTrain(step_decay)
model_wo_softmax = innvestigate.model_wo_softmax(best_model)
oneHot_NN = best_model.predict(X)

Train on 8000 samples, validate on 2000 samples


2024-08-26 13:23:48.988727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:48.990792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:48.992596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:48.994449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-26 13:23:48.996254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Epoch 1/100
5248/8000 [==================>...........] - ETA: 0s - loss: 0.2816 - mae: 0.2381 
Epoch 1: val_loss improved from inf to 0.08306, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5
8000/8000 [==============================] - 1s 118us/sample - loss: 0.2219 - mae: 0.1909 - val_loss: 0.0831 - val_mae: 0.0786 - lr: 0.0100
Epoch 2/100
 128/8000 [..............................] - ETA: 0s - loss: 0.0832 - mae: 0.0789

2024-08-26 13:23:49.991693: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


5632/8000 [====================>.........] - ETA: 0s - loss: 0.0680 - mae: 0.0648
Epoch 2: val_loss improved from 0.08306 to 0.04389, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5
8000/8000 [==============================] - 0s 13us/sample - loss: 0.0625 - mae: 0.0597 - val_loss: 0.0439 - val_mae: 0.0424 - lr: 0.0100
Epoch 3/100
5376/8000 [===================>..........] - ETA: 0s - loss: 0.0410 - mae: 0.0398
Epoch 3: val_loss improved from 0.04389 to 0.03037, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5
8000/8000 [==============================] - 0s 14us/sample - loss: 0.0386 - mae: 0.0374 - val_loss: 0.0304 - val_mae: 0.0296 - lr: 0.0100
Epoch 4/100
5376/8000 [===================>..........] - ETA: 0s - loss: 0.0295 - mae: 0.0287
Epoch 4: val_loss improved from 0.03037 to 0.02336, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model_simpleTests_LRP.h5
8000/8000 [===========================

In [4]:
idx = 2
x = X[np.newaxis,idx]
y_true = model_wo_softmax.predict(x)
pred_class = np.argmax(y_true[0])
W = model_wo_softmax.layers[1].get_weights()[0]
b = model_wo_softmax.layers[1].get_weights()[1]
R_last = y_true[0,pred_class]
x@W+b-y_true, y_true, W, b, x

(array([[-1.54702673e-04,  1.38792155e-05]]),
 array([[-0.5795374,  4.0242043]], dtype=float32),
 array([[-1.695598 ,  1.2197995],
        [-2.0202453,  1.2793059]], dtype=float32),
 array([-0.      ,  3.634342], dtype=float32),
 array([[0.15601864, 0.15599452]]))

### Understanding LRP-A1B0

In [5]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax)
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0: {a}, {y_true[0,pred_class]-np.sum(a)}")

lrp.alpha_1_beta_0: [[0.19030009 0.1995633 ]], 3.634340763092041


In [6]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax)
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0_IB: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB: [[1.9639564 2.0595555]], 0.0006923675537109375


In [7]:
denominator = x[0,0]*np.max(W[0,pred_class],0)+x[0,1]*np.max(W[1,pred_class],0)+2*np.max(b[pred_class],0)
R_0 = R_last * (x[0,0]*np.max(W[0,pred_class],0)+np.max(b[pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*np.max(W[1,pred_class],0)+np.max(b[pred_class],0)) / denominator
print(f"Manual LRP-A1BO: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-A1BO: 2.009671057763945,2.0145331963864455, 0.0


In [8]:
denominator = x[0,0]*np.max(W[0,pred_class],0)+x[0,1]*np.max(W[1,pred_class],0)+np.max(b[pred_class],0)
R_0 = R_last * (x[0,0]*np.max(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*np.max(W[1,pred_class],0)) / denominator
print(f"Manual LRP-A1BO without bias in numerator and only one bias in denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-A1BO without bias in numerator and only one bias in denominator: 0.1903108061325644,0.19956402739573104, 3.634329420622095


In [9]:
denominator = x[0,0]*np.max(W[0,pred_class],0)+x[0,1]*np.max(W[1,pred_class],0)
R_0 = R_last * (x[0,0]*np.max(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*np.max(W[1,pred_class],0)) / denominator
print(f"Manual LRP-A1BO without bias in both numerator and denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-A1BO without bias in both numerator and denominator: 1.9643472463166156,2.0598570078337755, -4.440892098500626e-16


### Understanding LRP-W2

In [10]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='WSquare')
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0: [[1.9170341 2.1082344]], -0.001064300537109375


In [11]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax, input_layer_rule='WSquare')

a = Analyze.analyze(x)

print(f"lrp.alpha_1_beta_0_IB: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB: [[1.9170341 2.1082344]], -0.001064300537109375


In [12]:
denominator = W[0,pred_class]**2 + W[1,pred_class]**2
R_0 = R_last * W[0,pred_class]**2 / denominator
R_1 = R_last * W[1,pred_class]**2 / denominator
print(f"Manual LRP-WSquare: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-WSquare: 1.916335747629476,2.1078685065209144, 4.440892098500626e-16


### Understanding LRP-Bounded

In [13]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0 bounded: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0 bounded: [[1.9635723 2.0594282]], 0.0012035369873046875


In [14]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0_IB', model_wo_softmax, input_layer_rule='Bounded')
a = Analyze.analyze(x)
print(f"lrp.alpha_1_beta_0_IB bounded: {a}, {R_last-np.sum(a)}")

lrp.alpha_1_beta_0_IB bounded: [[1.9635723 2.0594282]], 0.0012035369873046875


In [15]:
low = -1
high = 1
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class] \
            -low*(np.max(W[0,pred_class],0)+np.max(W[1,pred_class],0)) \
            -high*(np.min(W[0,pred_class],0)+np.min(W[1,pred_class],0))
R_0 = R_last * (x[0,0]*W[0,pred_class]-low*np.max(W[0,pred_class],0)-high*np.min(W[0,pred_class],0)) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]-low*np.max(W[1,pred_class],0)-high*np.min(W[1,pred_class],0)) / denominator
print(f"Manual LRP-Bounded: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Bounded: 1.9643472463166178,2.0598570078337777, -4.884981308350689e-15


In [16]:
Analyze = innvestigate.create_analyzer('lrp.z', model_wo_softmax)
a = Analyze.analyze(x)
print(f"lrp.z: {a}, {R_last-np.sum(a)}")

lrp.z: [[0.19030009 0.1995633 ]], 3.634340763092041


In [17]:
### lrp.z_IB doesn't exist!
# Analyze = innvestigate.create_analyzer('lrp.z', model_wo_softmax, bias = False)
# a = Analyze.analyze(x)
# print(f"lrp.z: {a}, {R_last-np.sum(a)}")

In [18]:
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class]+b[pred_class]
R_0 = R_last * (x[0,0]*W[0,pred_class]) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]) / denominator
print(f"Manual LRP-Z without bias in numerator and only one bias in denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Z without bias in numerator and only one bias in denominator: 0.1903108061325644,0.19956402739573104, 3.634329420622095


In [19]:
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class]
R_0 = R_last * (x[0,0]*W[0,pred_class]) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]) / denominator
print(f"Manual LRP-Z without bias in both numerator and denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Z without bias in both numerator and denominator: 1.9643472463166156,2.0598570078337755, -4.440892098500626e-16


In [20]:
Analyze = innvestigate.create_analyzer('lrp.epsilon', model_wo_softmax)
a = Analyze.analyze(x)
print(f"lrp.epsilon: {a}, {R_last-np.sum(a)}")

lrp.epsilon: [[0.19030009 0.1995633 ]], 3.634340763092041


In [21]:
Analyze = innvestigate.create_analyzer('lrp.epsilon', model_wo_softmax, bias = False)
a = Analyze.analyze(x)
print(f"lrp.epsilon_IB: {a}, {R_last-np.sum(a)}")

lrp.epsilon_IB: [[1.9639564 2.0595555]], 0.0006923675537109375


In [22]:
epsilon = 1e-7
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class]+b[pred_class]+epsilon
R_0 = R_last * (x[0,0]*W[0,pred_class]) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]) / denominator
print(f"Manual LRP-Epsilon without bias in numerator and only one bias in denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Epsilon without bias in numerator and only one bias in denominator: 0.1903108014034271,0.19956402243665533, 3.6343294303103084


In [23]:
epsilon = 1e-7
denominator = x[0,0]*W[0,pred_class]+x[0,1]*W[1,pred_class]+epsilon
R_0 = R_last * (x[0,0]*W[0,pred_class]) / denominator
R_1 = R_last * (x[0,1]*W[1,pred_class]) / denominator
print(f"Manual LRP-Epsilon without bias in both numerator and denominator: {R_0},{R_1}, {R_last-R_0-R_1}")

Manual LRP-Epsilon without bias in both numerator and denominator: 1.964346742477999,2.059856479497705, 1.032174686610432e-06
